# Working with Atlas SWIR Camera from LUCID Vision Labs

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import time
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

from cameras.lucid_swir import LucidSWIR
from thorlabs_elliptec import ELLx

save_cnt = 0
cam = LucidSWIR()

In [ ]:
stage = ELLx("/dev/ttyUSB0")
stage_pos = 0
stage.move_absolute_raw(stage_pos, True)

In [ ]:
curr_exp = cam._get_node_value("ExposureTime")
cam.printTemperatureInfo()
cam.set_fps(20)
cam.set_gain(0)

fig, ax = plt.subplots(1, 1)

# viz_min_val = 16
# viz_max_val = 40000
while True:
    tic = time.time()
    img_np = cam.getNextImage()
    min_val, max_val = np.percentile(img_np, (1, 99))
    print(f'Image Stats: Min: {img_np.min():05d}, Max: {img_np.max():05d}, Mean: {img_np.mean():05.2f}, Std: {img_np.std():05.2f}, 1%: {min_val:05.2f}, 99%: {max_val:05.2f}\r', end='')
    img_np = (img_np - min_val) / (max_val - min_val) * 255
    img_np = np.clip(img_np, 0, 255).astype(np.uint8)
    # img_np = (img_np*1.0 - img_np.min()) / (img_np.max() - img_np.min()) * 255
    # img_np = img_np / 255
    # img_np = np.clip(img_np, 0, 255).astype(np.uint8)
    cv2.imshow("Dispay", img_np)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break
    # elif key == ord('s'):
    #     cv2.imwrite(f"saved_img_{save_cnt}.png", img_np)
    #     print(f"\nSaved Image")
    #     save_cnt += 1
    elif key == ord('n'):
        stage_pos += 31
        if stage_pos > 93:
            stage_pos = 93
        stage.move_absolute_raw(stage_pos, True)
    elif key == ord('p'):
        stage_pos -= 31
        if stage_pos < 0:
            stage_pos = 0
        stage.move_absolute_raw(stage_pos, True)
    elif key == ord('w'):
        curr_exp += 1000
        cam.set_exposure(curr_exp)
    elif key == ord('s'):
        curr_exp -= 1000
        cam.set_exposure(curr_exp)


    # plt.imshow(img_np)

    # display.clear_output(wait=True)
    # display.display(plt.gcf())
    # print(f'FPS: {1/(time.time() - tic)}')
    # plt.waitforbuttonpress(10)

cv2.destroyAllWindows()
# cam.device.stop_stream()